In [83]:
import scipy.io
import numpy as np
import tensorflow as tf
# tf.compat.v1.disable_eager_execution()
import keras
import matplotlib.pyplot as plt


In [84]:
def DataPreprocessing(data, ff):

    fmat = {0.1:data['sbsl_PINN'][0][0], 0.08:data['sbsl_PINN'][1][0], 0.06:data['sbsl_PINN'][2][0], 0.04:data['sbsl_PINN'][3][0], 0.02:data['sbsl_PINN'][4][0], 0.01:data['sbsl_PINN'][5][0], 0.005:data['sbsl_PINN'][6][0], 0.0:data['sbsl_PINN'][7][0]}
    fval = [0.1  , 0.08 , 0.06 , 0.04 , 0.02 , 0.01 , 0.005, 0.000]
    i = ff
    N = fmat[i].shape[1]
    U = np.zeros((N,4))
    U[:,0] = data['eta']
    U[:,1] = fmat[i][0,:]
    U[:,2] = fmat[i][1,:]
    U[:,3] = fmat[i][2,:]
    #U[:,4] = torch.from_numpy(np.array([0.06]))

    D = np.array(data['D'])
    M = np.array(data['M'])
    u = np.array(data['ubar'])
    dMdx = np.array(data['dMdx'])
    dudx = np.array(data['dudx'])
    alpha = np.array(data['alp'])
    eta1 = np.array(data['eta'])   

    meanflow = np.zeros((D.shape[1],7))
    meanflow[:,0] = D
    meanflow[:,1] = M
    meanflow[:,2] = u
    meanflow[:,3] = dMdx
    meanflow[:,4] = dudx
    meanflow[:,5] = alpha
    meanflow[:,6] = eta1
    #meanflow = torch.tile(meanflow, (8,1))

    he = np.array([0.00])
    HE = np.tile(he, (N,1)) 
    #He = HE.flatten()[:,None]

    uu = U[:, 1:4]
    uu0 = U[:,0]
    uu0 = uu0.flatten()[:, None]

    input_set = np.concatenate([HE, uu0],1)

    return input_set, uu, meanflow

In [85]:
data = scipy.io.loadmat('/Users/ramtarun/Desktop/Cambridge/Indirect-Noise-in-Nozzles/Data/Data_PINN_subsonic_geom_linvelsup_f0-0.1.mat')
inputs, targets, meanflow = DataPreprocessing(data, ff=0.01)

In [86]:

def split_data(U, b_size, n_batches):
    '''
    Splits the data in batches. Each batch is created by sampling the signal with interval
    equal to n_batches.
    '''
    data   = np.zeros((n_batches, b_size, U.shape[1]), dtype=float)    
    for j in range(n_batches):
        data[j,:b_size] = U[::skip][j::n_batches].copy()

    return data

In [87]:
skip = 2
b_size = 30
n_batches = 5
val_batches = n_batches//2   #validation batches

## Training Data

In [88]:
x_tt = inputs[:b_size*n_batches*skip].copy()
y_tt = targets[:b_size*n_batches*skip].copy()
m_tt = meanflow[:b_size*n_batches*skip].copy()

Y_train     = split_data(y_tt, b_size, n_batches).astype(dtype=np.float32)
X_train     = split_data(x_tt, b_size, n_batches).astype(dtype=np.float32)
Meanflow_train = split_data(m_tt, b_size, n_batches).astype(dtype=np.float32)

## Validation Data 

In [89]:
x_vv        = inputs[b_size*n_batches*skip:b_size*n_batches*skip+b_size*val_batches*skip:].copy()
Y_vv        = targets[b_size*n_batches*skip:b_size*n_batches*skip+b_size*val_batches*skip:].copy()
meanflow_vv = meanflow[b_size*n_batches*skip:b_size*n_batches*skip+b_size*val_batches*skip:].copy()
Y_val       = split_data(Y_vv, b_size, val_batches).astype(dtype=np.float32) 
X_val       = split_data(x_vv, b_size, val_batches).astype(dtype=np.float32)
Meanflow_val = split_data(meanflow_vv, b_size, val_batches).astype(dtype=np.float32)

In [90]:
def RHS_ff_t(y, baseflow, f):

    pi_p, pi_m, sig = tf.unstack(y, axis=1)

    D = baseflow[:,0]
    M = baseflow[:,1]
    u = baseflow[:,2]
    dMdx = baseflow[:,3]
    dudx = baseflow[:,4]
    alpha = baseflow[:,5]
    eta1 = baseflow[:,6]    
    
    He = 0#a constant input

    gamma = 1.4#constant
    
    Msq = tf.math.square(M)

    Lambda = 1 + Msq * (gamma-1)/2
    zeta = f*gamma*Msq - 2*tf.math.tan(alpha)
    C1= ((gamma - 1)*(1-Msq)*f)/(2*Lambda*zeta)
    Ca = -C1*M*u*dMdx*(2-(2*Msq/(1-Msq)) - (2*gamma*Msq*(-2*f*Lambda - (gamma-1)/gamma *zeta)/(2*Lambda*zeta)))
    Ff = -(dudx + (4*f*u/(2*D)))

    denom = (M**2*u - u + C1*Msq *u + C1*Msq*Msq*gamma*u) #M**4
    vrh_p = M*(2*(1-M) + C1*M*(M-2+M*gamma*(1-2*M)))/(2*denom)
    vrh_m = -M*(2*(1+M) + C1*M*(M+2+M*gamma*(1+2*M)))/(2*denom)
    vkp_p = Msq*C1*(2+M*(gamma-1))/(2*denom)
    vkp_m = Msq*C1*(2-M*(gamma-1))/(2*denom)
    vth_p = C1*M*(M*(1+gamma) + M**2*(1-gamma) - 2)/denom
    vth_m = C1*M*(M*(1+gamma) - M**2*(1-gamma) + 2)/denom
    vsig = -(C1*Msq*(1+gamma*Msq) + Msq - 1)/denom
    calM = dMdx/(2*M)
    kp_p = (gamma - 1) + (2/M)
    kp_m = (gamma - 1) - (2/M)
    Gm_p = M*(Ca*(M + 1) + Ff*M*(C1*gamma*M*Msq + M + (1 - C1*Msq)))/(2*denom)
    Gm_m = M*(Ca*(M - 1) - Ff*M*(C1*gamma*M*Msq + M - (1 - C1*Msq)))/(2*denom)
    Ups = (Ca*(Msq - 1) - C1*Ff*Msq*Msq *(1+gamma))/denom
    
#     eq1 = - (2*np.pi*1j*He*vrh_p + Gm_m*kp_m + calM)*pi_p + (2*np.pi*1j*He*vkp_p + Gm_m*kp_p + calM)*pi_m - Gm_m*sig
#     eq2 = - (2*np.pi*1j*He*vkp_m + Gm_p*kp_m - calM)*pi_p - (2*np.pi*1j*He*vrh_m + Gm_p*kp_p + calM)*pi_m - Gm_m*sig
#     eq3 = - (2*np.pi*1j*He*vth_p + Ups*kp_m)*pi_p - (2*np.pi*1j*He*vth_m + Ups*kp_p)*pi_m - (2*np.pi*1j*He*vsig + Ups)*sig
    
#     eq1 = - tf.multiply((Gm_m*kp_m + calM),pi_p) + tf.multiply(( Gm_m*kp_p + calM),pi_m) - tf.multiply(Gm_m,sig)
    eq1 =  (Gm_m*kp_m + calM)*pi_p + ( Gm_m*kp_p - calM)*pi_m + Gm_m*sig
    eq2 =  ( Gm_p*kp_m - calM)*pi_p + ( Gm_p*kp_p + calM)*pi_m + Gm_m*sig
    eq3 =  ( Ups*kp_m)*pi_p + ( Ups*kp_p)*pi_m + (Ups)*sig
    



    return tf.stack([-eq1, -eq2, -eq3],axis=1)

In [93]:
### Model
model = keras.Sequential(name='PhyLSTM')
model.add(keras.layers.Dense(4, activation='linear', input_shape=(None, 2)))
# model.add(keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True,input_shape=(None, 2)))
model.add(keras.layers.Dense(4, activation='relu')) 
model.add(keras.layers.Dense(4, activation='tanh'))
model.add(keras.layers.Dense(4, activation='relu'))
model.add(keras.layers.Dense(4, activation='tanh'))
model.add(keras.layers.Dense(4, activation='sigmoid'))    

# xtrain = tf.expand_dims(X_train, axis=1)
print(model(X_train[0]))

model.summary()

tf.Tensor(
[[0.5        0.5        0.5        0.5       ]
 [0.49974158 0.5012792  0.5007597  0.49872464]
 [0.49948317 0.5025582  0.50151926 0.49744937]
 [0.4992248  0.50383687 0.5022788  0.4961744 ]
 [0.49896657 0.5051152  0.5030381  0.49489987]
 [0.4987085  0.50639296 0.5037971  0.49362582]
 [0.4984507  0.50767004 0.5045558  0.49235243]
 [0.49819303 0.50894636 0.5053142  0.4910799 ]
 [0.49793562 0.51022166 0.50607216 0.48980832]
 [0.49767864 0.5114959  0.50682956 0.48853773]
 [0.49742195 0.5127689  0.5075864  0.4872684 ]
 [0.49716562 0.5140406  0.5083426  0.4860003 ]
 [0.49690977 0.51531076 0.5090982  0.4847338 ]
 [0.49665427 0.51657933 0.509853   0.48346886]
 [0.4963994  0.5178462  0.51060694 0.48220557]
 [0.49614504 0.5191111  0.51136005 0.48094416]
 [0.49589127 0.52037406 0.51211226 0.47968474]
 [0.49563807 0.5216349  0.5128635  0.47842744]
 [0.49538553 0.52289337 0.51361364 0.47717234]
 [0.49513367 0.52414954 0.5143627  0.4759197 ]
 [0.49488258 0.52540314 0.51511055 0.4746695 ]
 [

In [94]:
@tf.custom_gradient
def DiffEq(x, baseflow, f):
    
    with tf.GradientTape() as tape:
        n = x[:,1]
        r = model(X_train[0])

    dy_dn = tape.gradient(r,n)
    gr = RHS_ff_t(r, baseflow, f)
    pde = dy_dn  + gr
    return pde

DiffEq(X_train[0], Meanflow_train[0], f=0.01)

AttributeError: 'numpy.ndarray' object has no attribute '_id'

In [43]:
class CustomModel(keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.mae_metric = keras.metrics.MeanSquaredError(name="mse")

    def train_step(self, xtrain, ytrain, baseflowtrain):
        x = xtrain
        y = ytrain
        bf = baseflowtrain
        eta = xtrain[:,1]

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute our own loss
            r = y_pred[:,:-1]
            f = tf.reduce_mean(y_pred[:,-1])
            loss1 = keras.losses.mean_squared_error(y, r)
            loss2 = DiffEq(r, eta, bf , f)
            loss = loss1 + loss2

        # Compute gradients
        trainable_vars = self.trainable_variables + [f]
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Compute our own metrics
        self.loss_tracker.update_state(loss)
        self.mae_metric.update_state(y, r)
        return {"loss": self.loss_tracker.result(), "mse": self.mae_metric.result()}

    @property
    def metrics(self):
        # We list our `Metric` objects here so that `reset_states()` can be
        # called automatically at the start of each epoch
        # or at the start of `evaluate()`.
        # If you don't implement this property, you have to call
        # `reset_states()` yourself at the time of your choosing.
        return [self.loss_tracker, self.mae_metric]
# Construct an instance of CustomModel
# inputs = keras.Input(shape=(32,))
# outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(model)

# We don't passs a loss or metrics here.
model.compile(optimizer="adam")

# # Just use `fit` as usual -- you can use callbacks, etc.
# x = np.random.random((1000, 32))
# y = np.random.random((1000, 1))
model.fit(X_train, Y_train, Meanflow_train, epochs=5)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [11]:
def loss_fn(outs, targets, pde):
    l1 = tf.keras.losses.MSE(targets, outs)
    l2 = tf.reduce_mean(tf.square(pde))
    return l1+l2

In [12]:
optim = tf.keras.optimizers.legacy.Adam(amsgrad=True)

In [13]:
model.compile(loss = loss_fn, optimizer = optim)

In [16]:
model.fit(X_train, Y_train, Meanflow_train, epochs = 100, batch_size = 30)

TypeError: Model.fit() got multiple values for argument 'batch_size'

In [ ]:
@tf.function
def train(Xtrain, ytrain, baseflow, model, train = True):
    
    n = Xtrain[:,1]
    xtrain = tf.expand_dims(Xtrain, axis=1)
    outs = model(xtrain) [:,:3]
    f = tf.reduce_mean(outs[:,-1])
    print(f)
    PDE = DiffEq(outs, n, baseflow, f)
    print(PDE)
    trn_loss = loss_fn(outs, ytrain, pde=)
    


    if train:
        param = model.trainable_weights + [f]
        #compute and apply gradients
        grads   = tf.gradients(trn_loss, param)
        optim.apply_gradients(zip(grads, param))


    return  outs, trn_loss


In [ ]:
plt.rcParams["figure.figsize"] = (15,4)
plt.rcParams["font.size"]  = 20

Loss_Mse    = tf.keras.losses.MeanSquaredError()

n_epochs    = 5001 #number of epochs

#define optimizer and initial learning rate   
optimizer   = tf.keras.optimizers.legacy.Adam(amsgrad=True) #amsgrad True for better convergence
# optimizer   = tf.keras.optimizers.Adam #amsgrad True for better convergence

l_rate                  = 0.01
optimizer.learning_rate = l_rate

# quantities to check and store the training and validation loss and the training goes on
old_loss      = np.zeros(n_epochs) #needed to evaluate training loss convergence
tloss_plot    = np.zeros(n_epochs) #training loss
vloss_plot    = np.zeros(n_epochs) #validation loss
tloss1_plot   = np.zeros(n_epochs) #training_der loss
vloss1_plot   = np.zeros(n_epochs) #validation_der loss
old_loss[0]  = 1e6 #initial value has to be high
N_check      = 5   #each N_check epochs we check convergence and validation loss
patience     = 200 #if the val_loss has not gone down in the last patience epochs, early stop
last_save    = patience

t            = 1 # initial (not important value) to monitor the time of the training

for epoch in range(n_epochs):
    
    if epoch - last_save > patience: break #early stop
                
    #Perform gradient descent for all the batches every epoch
    loss_training = 0
#     rng.shuffle(t_train, axis=0) #shuffle batches

    for j in range(n_batches-2):
        temp = train(X_train[j], Y_train[j], Meanflow_train[j], model)
        # loss_training += loss
        print(temp)
    
#     #save training loss each epoch
   
    # tloss_plot[epoch]  = loss_training/n_batches
        
    # if (epoch%100==0) and epoch != 0: 
    #     f = tf.mean(outs[:,-1])
    #     print(f.numpy())
    #     #plot convergence of training and validation loss
    #     plt.subplot(1,2,1)
    #     plt.title('MSE convergence')
    #     plt.yscale('log')
    #     plt.grid(True, axis="both", which='both', ls="-", alpha=0.3)
    #     plt.plot(tloss_plot[np.nonzero(tloss_plot)], 'g', label='Train loss')

    #     plt.xlabel('epochs')
    #     plt.subplot(1,2,2)
    #     plt.plot(Y_train[-1], 'black')
    #     plt.plot(outs[:,:3].numpy(), 'b--')

        
    #     plt.tight_layout()
    #     plt.show()